In [3]:
import os
import glob
import rasterio
from rasterio.features import geometry_mask
from shapely.geometry import shape
import numpy as np
import matplotlib.pyplot as plt
from rasterio.merge import merge
from rasterio.transform import from_origin

## Merging all tif images into one tif image

### Create the variable "tif_paths" that contain all image directories

In [4]:
# Folder containing GeoTIFF images
folder_path = r"C:\Users\yixiu\data_from_eosloan_reloc\historical_data\all_tiles_historical_resampled_500m\16bit_20060529"

# Define the search pattern for GeoTIFF files
tif_pattern = os.path.join(folder_path, "*.tif")

# Use glob to get a list of all GeoTIFF files in the folder
tif_paths = glob.glob(tif_pattern)

# Print the list of GeoTIFF paths
for tif_path in tif_paths:
    print(tif_path)

C:\Users\yixiu\data_from_eosloan_reloc\historical_data\all_tiles_historical_resampled_500m\16bit_20060529\QB2_06MAY29055547-MS-017137790010_01_P001_resampled_grid500_1083.tif
C:\Users\yixiu\data_from_eosloan_reloc\historical_data\all_tiles_historical_resampled_500m\16bit_20060529\QB2_06MAY29055547-MS-017137790010_01_P001_resampled_grid500_1184.tif
C:\Users\yixiu\data_from_eosloan_reloc\historical_data\all_tiles_historical_resampled_500m\16bit_20060529\QB2_06MAY29055547-MS-017137790010_01_P001_resampled_grid500_1185.tif
C:\Users\yixiu\data_from_eosloan_reloc\historical_data\all_tiles_historical_resampled_500m\16bit_20060529\QB2_06MAY29055547-MS-017137790010_01_P001_resampled_grid500_1285.tif
C:\Users\yixiu\data_from_eosloan_reloc\historical_data\all_tiles_historical_resampled_500m\16bit_20060529\QB2_06MAY29055547-MS-017137790010_01_P001_resampled_grid500_1286.tif
C:\Users\yixiu\data_from_eosloan_reloc\historical_data\all_tiles_historical_resampled_500m\16bit_20060529\QB2_06MAY29055547-M

### Merge all the images

In [5]:
# Open all GeoTIFFs
src_files_to_mosaic = [rasterio.open(path) for path in tif_paths]

# Merge images
mosaic, out_trans = merge(src_files_to_mosaic)

# Create an output GeoTIFF
out_meta = src_files_to_mosaic[0].meta.copy()
out_meta.update({"driver": "GTiff",
                 "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans})

# Write the mosaic to a new GeoTIFF file
with rasterio.open(r"C:\Users\yixiu\data_from_eosloan_reloc\historical_data\all_tiles_historical_resampled_500m\16bit_20060529_combined\output_mosaic.tif", "w", **out_meta) as dest:
    dest.write(mosaic)

## Determine overlapping area

### Create the function to create binary mask

In [ ]:
def create_binary_mask(image_path):
    with rasterio.open(image_path) as src:
        # Get the full geometry of the image
        full_geometry = src.shape

        # Create a binary mask for the entire image
        mask = geometry_mask([full_geometry], transform=src.transform, invert=False, out_shape=src.shape, dtype='bool')
        return mask

### Combine all three masks

In [ ]:
def combine_masks(mask1, mask2, mask3):
    # Combine three binary masks using logical AND
    combined_mask = np.logical_and(np.logical_and(mask1, mask2), mask3)
    return combined_mask

### Apply the combined mask to determine overlapping area

In [ ]:
def apply_combined_mask(image_path, combined_mask):
    with rasterio.open(image_path) as src:
        image_data = src.read(1)

        # Apply the combined binary mask
        masked_data = np.where(combined_mask, image_data, 0)

        return masked_data

### Plot the original and the masked images

In [ ]:
def plot_masked_image(image_path, masked_data):
    with rasterio.open(image_path) as src:
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))

        axes[0].imshow(src.read(1), cmap='gray')
        axes[0].set_title('Original Image')

        axes[1].imshow(masked_data, cmap='gray')
        axes[1].set_title('Masked Image')

        plt.show()

In [ ]:
if __name__ == "__main__":
    # Provide paths to your three raster images
    image_path1 = "C:\Users\yixiu\data_from_eosloan_reloc\historical_data\all_tiles_historical_resampled_500m\16bit_20060529_combined\output_mosaic_20060529.tif"
    image_path2 = "C:\Users\yixiu\data_from_eosloan_reloc\historical_data\all_tiles_historical_resampled_500m\16bit_20060529_combined\output_mosaic_20080420.tif"
    image_path3 = "C:\Users\yixiu\data_from_eosloan_reloc\historical_data\all_tiles_historical_resampled_500m\16bit_20060529_combined\output_mosaic_20100413.tif"

    # Create binary masks for each image
    mask1 = create_binary_mask(image_path1)
    mask2 = create_binary_mask(image_path2)
    mask3 = create_binary_mask(image_path3)

    # Combine the masks to get the common overlapping area
    combined_mask = combine_masks(mask1, mask2, mask3)

    # Apply the combined mask to each image
    masked_data1 = apply_combined_mask(image_path1, combined_mask)
    masked_data2 = apply_combined_mask(image_path2, combined_mask)
    masked_data3 = apply_combined_mask(image_path3, combined_mask)
    
    # Now, masked_data1, masked_data2, and masked_data3 contain the overlapping areas of the three images
    
    # Plot the original images and the masked images
    plot_masked_image(image_path1, masked_data1)
    plot_masked_image(image_path2, masked_data2)
    plot_masked_image(image_path3, masked_data3)